In [26]:
# Importar as bibliotecas
import pandas as pd 
import numpy as np

ENDERECO_DADOS = 'https://www.ispdados.rj.gov.br/Arquivos/BaseDPEvolucaoMensalCisp.csv'

# Obtendo dados
try:
    df_roubos = pd.read_csv(ENDERECO_DADOS, sep=';', encoding='iso-8859-1')
    # print(df_roubos['munic'].unique())  # testar as impressões
    # print(df_roubos['regiao'].unique())  # testar as impressões
    print(df_roubos.head())
except Exception as e:
    print("Erro ao obter dados do ISP: ", e)




   cisp  mes   ano  mes_ano  aisp  risp           munic    mcirc   regiao  \
0     1    1  2003  2003m01     5     1  Rio de Janeiro  3304557  Capital   
1     4    1  2003  2003m01     5     1  Rio de Janeiro  3304557  Capital   
2     5    1  2003  2003m01     5     1  Rio de Janeiro  3304557  Capital   
3     6    1  2003  2003m01     1     1  Rio de Janeiro  3304557  Capital   
4     7    1  2003  2003m01     1     1  Rio de Janeiro  3304557  Capital   

   hom_doloso  ...  cmp  cmba  ameaca  pessoas_desaparecidas  \
0           0  ...  NaN   NaN      21                      2   
1           3  ...  NaN   NaN      15                      6   
2           3  ...  NaN   NaN      47                      2   
3           6  ...  NaN   NaN      26                      2   
4           4  ...  NaN   NaN      10                      1   

   encontro_cadaver  encontro_ossada  pol_militares_mortos_serv  \
0                 0                0                          0   
1                 

In [27]:
# Tratando e preparando os dados
try:
    # Delimitando as variáveis
    df_roubos = df_roubos[['cisp', 'regiao', 'munic', 'roubo_veiculo']]
    # print(df_roubos.head())
    # Tratando os erros de acentuação na série região
    correcao_acentuacao = df_roubos['regiao'].str.startswith('Grande Niter', na=False)
    # print(correcao_acentuacao)
    df_roubos.loc[correcao_acentuacao, 'regiao'] = 'Grande Niterói'
    # print(df_roubos['regiao'].unique())

    # Agrupando os dados cisp, região e município
    df_roubos = df_roubos.groupby(['cisp', 'regiao', 'munic']).sum(['roubo_veiculo']).reset_index()

    # Display() é uma função do Jupyter Notebook que permite exibir o DataFrame
    display(df_roubos)
except Exception as e:
    print("Erro no tratamento dos dados: ", e)


,cisp,regiao,munic,roubo_veiculo
0,1,Capital,Rio de Janeiro,565
1,4,Capital,Rio de Janeiro,2165
2,5,Capital,Rio de Janeiro,1316
3,6,Capital,Rio de Janeiro,4516
4,7,Capital,Rio de Janeiro,1997
...,...,...,...,...
141,159,Interior,Cachoeiras de Macacu,283
142,165,Interior,Mangaratiba,500
143,166,Interior,Angra dos Reis,968
144,167,Interior,Paraty,105


In [28]:
# Calculando as medidas
try:
    array_roubos = np.array(df_roubos['roubo_veiculo'])
    # print(array_roubos)
    
    media = np.mean(array_roubos)
    mediana = np.median(array_roubos)
    total = np.sum(array_roubos)
    # maximo = np.max(array_roubos)
    # minimo = np.min(array_roubos)

    # Obtendo os Quartis
    q1 = np.quantile(array_roubos, 0.25)
    q3 = np.quantile(array_roubos, 0.75)

    # Printando as medidas
    print('Medidas:')
    print(f'Media: {media:.2f}')
    print(f'Mediana: {mediana}')
    print(f'Total: {total}')

except Exception as e:
    print("Erro ao obter as medidas: ", e)



Medidas:
Media: 4778.64
Mediana: 928.0
Total: 697681


In [29]:
# Identificando os maiores e menores
try:
    # Gerar um dataframe com os maiores
    # Copy() quando preciar alterar um dataframe já filtrado
    df_maiores =  df_roubos[df_roubos['roubo_veiculo'] > q3].copy()
    df_maiores['flag'] = 'mais'
    # print(df_maiores)

    # Gerar um dataframe com os menores
    df_menores = df_roubos[df_roubos['roubo_veiculo'] < q1].copy()
    df_menores['flag'] = 'menos'
    # print(df_menores)

    # Concatenar os dois dataframes
    df_roubos_flags = pd.concat([df_maiores, df_menores], ignore_index=True)

    display(df_roubos_flags)
except Exception as e:
    print("Erro ao identificar os maiores e menores: ", e)



,cisp,regiao,munic,roubo_veiculo,flag
0,17,Capital,Rio de Janeiro,7505,mais
1,20,Capital,Rio de Janeiro,8911,mais
2,21,Capital,Rio de Janeiro,19279,mais
3,22,Capital,Rio de Janeiro,14858,mais
4,23,Capital,Rio de Janeiro,10612,mais
...,...,...,...,...,...
69,155,Interior,São Sebastião do Alto,29,menos
70,156,Interior,Santa Maria Madalena,47,menos
71,157,Interior,Trajano de Morais,20,menos
72,158,Interior,Bom Jardim,21,menos


In [30]:
# Exportando dados csv ou xlsx
try:
    # Testar sem encoding | Impirmir com o encoding utf-8-sig, se precisar do csv fora do BI
    df_roubos_flags.to_csv('roubos_veiculos.csv', index=False, )

    # df_roubos_flags.to_excel('roubos_veiculos.xlsx', index=False)

except Exception as e:
    print("Erro ao exportar os dados: ", e)


In [32]:
# Jogar o dataframe como uma tabela no Banco de dados
from sqlalchemy import create_engine, text

try:
    host = 'localhost'
    user = 'root'
    password = ''
    database = 'bd_roubos_flags'

    engine = create_engine(f'mysql+pymysql://{user}:{password}@{host}/{database}')

    tabela = 'tb_roubo_veiculos'

    # Na primeira vez, usar o 'replace' ou 'fail' para criar a tabela
    # A partir da segunda vez, usar o 'append' para adicionar os dados.
    df_roubos_flags.to_sql(tabela, con=engine, if_exists='fail', index=False)

except Exception as e:
    print("Erro ao exportar os dados: ", e)

Erro ao exportar os dados:  Table 'tb_roubo_veiculos' already exists.


In [ ]:
# READ
# Estrutura with para abrir e fechar o arquivo
with engine.connect() as conexao:
    comando_sql = text("SELECT * FROM tb_roubo_veiculos")
    resultado = conexao.execute(comando_sql)
    for linha in resultado:
        print(linha)

In [34]:
# CREATE
with engine.connect() as conexao:
    comando_sql = text(
        """
            INSERT INTO tb_roubo_veiculos (cisp, regiao, munic, roubo_veiculo, flag)
            VALUES (171, 'Grande Niterói', 'São Gonçalo', 500, 'mais')
        """
    )
    
    conexao.execute(comando_sql)

    conexao.commit()

In [37]:
# CREATE
# Estrutura with para abrir e fechar o arquivo
with engine.connect() as conexao:
    comando_sql = text(
    # Inserir a 69 também ('Niterói', 'Grande Niterói', 'mais')
        """
        INSERT INTO tb_roubo_veiculos (cisp, regiao, munic, roubo_veiculo, flag)
        VALUES (:cisp, :regiao, :munic, :roubo_veiculo, :flag)
        """
    )
    
    conexao.execute(comando_sql, {
        'cisp': 69,
        'regiao': 'Grande Niterói',
        'munic': 'São Gonçalo',
        'roubo_veiculo': 100,
        'flag': 'menos'
    })

    conexao.commit()


In [36]:
# UPDATE
# Onde a cisp é 171, passa a ser 76,
# e atualizando para São Gonçalo, flag 'menos', roubo 20 etc
with engine.connect() as conexao:

    regiao = 'Grande Niterói'
    munic = 'Niterói'
    roubo_veiculo = 20
    flag = 'menos'
    cisp_nova = 76
    cisp_atual = 171

    comando_sql = text(
        """
            UPDATE tb_roubo_veiculos
            SET regiao = :regiao,
                cisp = :cisp_nova,
                munic = :munic,
                roubo_veiculo = :roubo_veiculo,
                flag = :flag
            WHERE cisp = :cisp_atual
        """
    )

    conexao.execute(comando_sql, {
        'regiao': regiao,
        'cisp_nova': cisp_nova,
        'munic': munic,
        'roubo_veiculo': roubo_veiculo,
        'flag': flag,
        'cisp_atual': cisp_atual
        }
    )
    
    conexao.commit()

In [39]:
 # DELETE
with engine.begin() as conexao: #engine.connect()

    comando_sql = text(
        """
            DELETE FROM tb_roubo_veiculos
            WHERE cisp = :cisp
        """
    )

    conexao.execute(comando_sql, {
        'cisp': 76
        }
    )

    #conexao.commit()